In [5]:
import sympy as sp
from sympy.physics.mechanics import dynamicsymbols, mechanics_printing
from sympy.printing.octave import octave_code

mechanics_printing()

class RobotMathEngine6DOF:
    def __init__(self, joint_config, link_vectors_mask):
        self.joint_config = joint_config
        # Converte para Matrix para garantir precisão e evitar floats
        self.link_vectors_mask = [sp.Matrix(v) for v in link_vectors_mask]

        self.t = sp.symbols('t')
        self.g = sp.symbols('g')

        self.q = []
        self.dq = []
        self.params_list = []

        self.frames = []
        self.rotation_matrices = []
        self.com_positions_global = []
        self.angular_velocities = []
        self.masses = []

        self.M = None
        self.G_vec = None
        self.C_total = None # Vetor ÚNICO (Muito mais rápido)
        self.Jacobian = None
        self.M_cse_replacements = []
        self.M_cse = None

    def _rot_matrix_local(self, axis, angle):
        c, s = sp.cos(angle), sp.sin(angle)
        if axis == 'x': return sp.Matrix([[1,0,0],[0,c,-s],[0,s,c]])
        if axis == 'y': return sp.Matrix([[c,0,s],[0,1,0],[-s,0,c]])
        if axis == 'z': return sp.Matrix([[c,-s,0],[s,c,0],[0,0,1]])
        return sp.eye(3)

    def _get_axis_vector(self, axis):
        if axis == 'x': return sp.Matrix([1, 0, 0])
        if axis == 'y': return sp.Matrix([0, 1, 0])
        if axis == 'z': return sp.Matrix([0, 0, 1])
        return sp.Matrix([0,0,0])

    def step_1_kinematics(self):
        print("1. Cinemática (Matrizes de Rotação e CMs)...")
        T_acc = sp.eye(4)
        R_acc = sp.eye(3)
        omega_acc = sp.Matrix([0, 0, 0])

        for i, (j_type, link_vec) in enumerate(zip(self.joint_config, self.link_vectors_mask)):
            q = dynamicsymbols(f'q{i+1}')
            dq = dynamicsymbols(f'q{i+1}', 1)
            self.q.append(q)
            self.dq.append(dq)

            m = sp.symbols(f'm{i+1}')
            L = sp.symbols(f'L{i+1}')
            cx, cy, cz = sp.symbols(f'cx{i+1} cy{i+1} cz{i+1}')
            self.masses.append(m)
            self.params_list.extend([m, L, cx, cy, cz])

            type_char, axis_char = j_type[0], j_type[1].lower()
            axis_vec_local = self._get_axis_vector(axis_char)

            # Rotação
            if type_char == 'R':
                axis_vec_global = R_acc * axis_vec_local
                omega_new = omega_acc + axis_vec_global * dq
                R_j = self._rot_matrix_local(axis_char, q)
                P_j = sp.Matrix([0,0,0])
            elif type_char == 'D':
                omega_new = omega_acc
                R_j = sp.eye(3)
                P_j = sp.Matrix([0,0,0])
                if axis_char == 'x': P_j[0] = q
                if axis_char == 'y': P_j[1] = q
                if axis_char == 'z': P_j[2] = q

            R_acc = R_acc * R_j
            self.rotation_matrices.append(R_acc)
            self.angular_velocities.append(omega_new)
            omega_acc = omega_new

            # Geometria
            T_joint = sp.eye(4)
            T_joint[0:3, 0:3] = R_j
            T_joint[0:3, 3] = P_j
            T_at_joint_start = T_acc * T_joint

            # CM
            v_com_local = sp.Matrix([cx, cy, cz, 1])
            p_com_global = T_at_joint_start * v_com_local
            self.com_positions_global.append(p_com_global[0:3, :])

            # Elo
            P_link_vec = link_vec * L
            T_link = sp.eye(4)
            T_link[0:3, 3] = P_link_vec
            T_acc = T_at_joint_start * T_link
            self.frames.append(T_acc)

    def step_2_jacobian_M_G(self):
        print("2. Dinâmica M e G (Método Jacobianos Rápido)...")
        n = len(self.q)
        self.M = sp.zeros(n, n)
        V_tot = 0

        for i in range(n):
            m = self.masses[i]
            pos_cm = self.com_positions_global[i]
            R_global = self.rotation_matrices[i]

            J_v = pos_cm.jacobian(self.q)
            J_w = self.angular_velocities[i].jacobian(self.dq)

            Ixx, Iyy, Izz = sp.symbols(f'Ixx{i+1} Iyy{i+1} Izz{i+1}')
            self.params_list.extend([Ixx, Iyy, Izz])
            I_local = sp.Matrix([[Ixx, 0, 0], [0, Iyy, 0], [0, 0, Izz]])
            I_global = R_global * I_local * R_global.T

            self.M += m * J_v.T * J_v + J_w.T * I_global * J_w
            V_tot += m * self.g * pos_cm[2]

        self.G_vec = sp.Matrix([V_tot]).jacobian(self.q).T
        self.Jacobian = self.frames[-1][0:3, 3].jacobian(self.q)

        print("   Compactando M com CSE para reuso nas derivadas...")
        replacements, reduced = sp.cse(self.M)
        self.M_cse_replacements = replacements
        self.M_cse = reduced[0] if reduced else self.M

    def step_3_coriolis_combined(self):
        print("3. Dinâmica Coriolis Total (Sem separação)...")
        n = len(self.q)
        self.C_total = sp.zeros(n, 1)

        # OTIMIZAÇÃO: Não calculamos a matriz C(n,n), calculamos o VETOR V(n,1)
        # V_i = Sum_j Sum_k ( c_ijk * dq_j * dq_k )

        print(f"   Calculando vetor de {n} linhas...")

        if self.M_cse_replacements:
            print("   Reaproveitando CSE de M nas derivadas...")
            cse_sym_to_func = {}
            cse_derivative_subs = {}
            for sym, expr in self.M_cse_replacements:
                func = sp.Function(str(sym))(*self.q)
                cse_sym_to_func[sym] = func
                for qk in self.q:
                    cse_derivative_subs[sp.diff(func, qk)] = sp.diff(expr, qk)
            M_for_diff = self.M_cse.xreplace(cse_sym_to_func)
            dM_dq = [sp.Matrix(M_for_diff).diff(qk) for qk in self.q]
            func_to_expr = {func: expr for sym, func in cse_sym_to_func.items()}
            dM_dq = [mat.xreplace(cse_derivative_subs).xreplace(func_to_expr) for mat in dM_dq]
        else:
            dM_dq = [self.M.diff(qk) for qk in self.q]

        for i in range(n):
            termo_linha = 0
            for j in range(n):
                for k in range(j, n):
                    # Símbolo de Christoffel do Primeiro Tipo
                    # c_ijk = 0.5 * ( dM_ij/dq_k + dM_ik/dq_j - dM_jk/dq_i )

                    # Dica: Só calcule se dq_j e dq_k forem relevantes para M_ij...
                    # Mas em Python puro é dificil checar dependencia rapido.
                    # Vamos na força bruta mas SEM SIMPLIFY.

                    dM_ij_dk = dM_dq[k][i, j]
                    dM_ik_dj = dM_dq[j][i, k]
                    dM_jk_di = dM_dq[i][j, k]

                    c_ijk = sp.Rational(1,2) * (dM_ij_dk + dM_ik_dj - dM_jk_di)

                    if c_ijk != 0:
                        termo = c_ijk * self.dq[j] * self.dq[k]
                        if k != j:
                            termo *= 2
                        termo_linha += termo

            self.C_total[i] = sp.collect(termo_linha, self.dq)
            print(f"   -> Linha {i+1} ok")

    def step_4_export_cse(self):
        print("\n" + "="*40)
        print(" OTIMIZANDO E EXPORTANDO (CSE) ")
        print("="*40)

        mapa_subs = {}
        for i in range(len(self.q)):
            sym_q = sp.Symbol(f'q{i+1}')
            sym_dq = sp.Symbol(f'dq{i+1}')
            mapa_subs[self.q[i]] = sym_q
            mapa_subs[self.dq[i]] = sym_dq

        # Agrupa expressões
        expressoes = [
            self.M.subs(mapa_subs),
            self.G_vec.subs(mapa_subs),
            self.C_total.subs(mapa_subs),
            self.Jacobian.subs(mapa_subs)
        ]

        print("   Compactando equações (Isso reduz o tamanho do código)...")
        replacements, reduced = sp.cse(expressoes)

        print("% --- Variáveis Auxiliares ---")
        for sym, expr in replacements:
            print(f"{sym} = {octave_code(expr)};")

        M_opt, G_opt, C_opt, J_opt = reduced

        print("\n% --- Matrizes Finais ---")
        print("M = " + octave_code(M_opt) + ";")
        print("G = " + octave_code(G_opt) + ";")
        print("V_total = " + octave_code(C_opt) + "; % Coriolis + Centripeta Juntos")
        print("J = " + octave_code(J_opt) + ";")

# --- RODAR PARA O ANTROPOMÓRFICO ---
antropo_joints = ['Rz', 'Ry', 'Ry', 'Rz', 'Ry', 'Rz']
antropo_links = [[0, 0, 1], [1, 0, 0], [1, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 1]]

bot_6dof = RobotMathEngine6DOF(antropo_joints, antropo_links)
bot_6dof.step_1_kinematics()
bot_6dof.step_2_jacobian_M_G()
bot_6dof.step_3_coriolis_combined()
bot_6dof.step_4_export_cse()


1. Cinemática (Matrizes de Rotação e CMs)...
2. Dinâmica M e G (Método Jacobianos Rápido)...
   Compactando M com CSE para reuso nas derivadas...
3. Dinâmica Coriolis Total (Sem separação)...
   Calculando vetor de 6 linhas...
   Reaproveitando CSE de M nas derivadas...
   -> Linha 1 ok
   -> Linha 2 ok
   -> Linha 3 ok
   -> Linha 4 ok
   -> Linha 5 ok
   -> Linha 6 ok

 OTIMIZANDO E EXPORTANDO (CSE) 
   Compactando equações (Isso reduz o tamanho do código)...
% --- Variáveis Auxiliares ---
x0 = sin(q2);
x1 = x0.^2;
x2 = cos(q3);
x3 = x0.*x2;
x4 = sin(q3);
x5 = cos(q2);
x6 = x4.*x5;
x7 = -x3 - x6;
x8 = x7.^2;
x9 = x5.^2;
x10 = x0.*x4;
x11 = x10 - x2.*x5;
x12 = -x11;
x13 = x12.^2;
x14 = cos(q1);
x15 = sin(q1);
x16 = cx2.*x5;
x17 = cz2.*x0;
x18 = -cy2.*x14 - x15.*x16 - x15.*x17;
x19 = -cy2.*x15 + x14.*x16 + x14.*x17;
x20 = L2.*x5;
x21 = x14.*x20;
x22 = x14.*x3 + x14.*x6;
x23 = -x10.*x14 + x14.*x2.*x5;
x24 = cx3.*x23 - cy3.*x15 + cz3.*x22 + x21;
x25 = x15.*x20;
x26 = x10.*x15 - x15.*x2.*

In [6]:
# --- VALIDAÇÃO FINAL ---
inputs_joints = ['Rz', 'Rz']
inputs_links = [[1, 0, 0], [1, 0, 0]]

# Instancia o Novo Motor Corrigido
bot = RobotMathEngine6DOF(inputs_joints, inputs_links)
bot.step_1_kinematics()
bot.step_2_jacobian_M_G()
bot.step_3_coriolis_combined()
bot.step_4_export_cse()

print("\n--- COMPARANDO COM NIKU (Frações Exatas) ---")
m1, m2, l1, l2 = sp.symbols('m1 m2 L1 L2')
C2 = sp.cos(bot.q[1])

# Matriz Alvo Niku (Exata)
M11_n = sp.Rational(1,3)*m1*l1**2 + m2*l1**2 + sp.Rational(1,3)*m2*l2**2 + m2*l1*l2*C2
M12_n = sp.Rational(1,3)*m2*l2**2 + sp.Rational(1,2)*m2*l1*l2*C2
M22_n = sp.Rational(1,3)*m2*l2**2

M_Niku = sp.Matrix([[M11_n, M12_n], [M12_n, M22_n]])

# Substituições
subs_map = {
    sp.symbols('L1'): l1, sp.symbols('m1'): m1,
    sp.symbols('L2'): l2, sp.symbols('m2'): m2,
    sp.symbols('cx1'): l1/2, sp.symbols('cx2'): l2/2, # CM no meio
    sp.symbols('cy1'): 0, sp.symbols('cz1'): 0,
    sp.symbols('cy2'): 0, sp.symbols('cz2'): 0,
    sp.symbols('Izz1'): sp.Rational(1,12)*m1*l1**2,
    sp.symbols('Izz2'): sp.Rational(1,12)*m2*l2**2,
    sp.symbols('Ixx1'): 0, sp.symbols('Iyy1'): 0,
    sp.symbols('Ixx2'): 0, sp.symbols('Iyy2'): 0
}

# Subtração
Diff = sp.simplify(sp.expand(bot.M.subs(subs_map) - M_Niku))

print("Resultado da Diferença:")
sp.pprint(Diff)

if Diff == sp.zeros(2,2):
    print("\n✅ SUCESSO! A validação bateu perfeitamente.")
else:
    print("\n❌ AINDA TEM ERRO.")

1. Cinemática (Matrizes de Rotação e CMs)...
2. Dinâmica M e G (Método Jacobianos Rápido)...
   Compactando M com CSE para reuso nas derivadas...
3. Dinâmica Coriolis Total (Sem separação)...
   Calculando vetor de 2 linhas...
   Reaproveitando CSE de M nas derivadas...
   -> Linha 1 ok
   -> Linha 2 ok

 OTIMIZANDO E EXPORTANDO (CSE) 
   Compactando equações (Isso reduz o tamanho do código)...
% --- Variáveis Auxiliares ---
x0 = cos(q1);
x1 = sin(q1);
x2 = L1.*x0;
x3 = sin(q2);
x4 = cos(q2);
x5 = -x0.*x4 + x1.*x3;
x6 = -x5;
x7 = -x0.*x3 - x1.*x4;
x8 = cx2.*x6 + cy2.*x7;
x9 = x2 + x8;
x10 = L1.*x1;
x11 = cx2.*x7 + cy2.*x5;
x12 = -x10 + x11;
x13 = Izz2 + m2.*x11.*x12 + m2.*x8.*x9;
x14 = L2.*x6;

% --- Matrizes Finais ---
M = [Izz1 + Izz2 + m1.*(cx1.*x0 - cy1.*x1).^2 + m1.*(-cx1.*x1 - cy1.*x0).^2 + m2.*x12.^2 + m2.*x9.^2 x13; x13 Izz2 + m2.*x11.^2 + m2.*x8.^2];
G = [0; 0];
V_total = [0; 0]; % Coriolis + Centripeta Juntos
J = [L2.*x7 - x10 L2.*x7; x14 + x2 x14; 0 0];

--- COMPARANDO COM N

In [8]:
# --- RODAR PARA O UVMS ---
# --- 1. CONFIGURAÇÃO DO VEÍCULO (BASE FLUTUANTE) ---
# O veículo possui 6 graus de liberdade livres.
# Modelamos isso como 3 juntas prismáticas (XYZ) e 3 rotativas (RPY).
# Nota: "D" = Prismática (Deslocamento), "R" = Rotativa

vehicle_joints = ['Dx', 'Dy', 'Dz', 'Rz', 'Ry', 'Rx']

# Os "elos" virtuais do veículo geralmente têm comprimento zero,
# pois todos os movimentos ocorrem em torno do centro de flutuabilidade/massa.
vehicle_links = [[0, 0, 0]] * 6

# --- 2. CONFIGURAÇÃO DO BRAÇO (ANTROPOMÓRFICO) ---
# A configuração que você passou:
arm_joints = ['Rz', 'Ry', 'Ry', 'Rz', 'Ry', 'Rz']
arm_links =  [[0, 0, 1], [1, 0, 0], [1, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 1]]

# --- 3. MONTAGEM DO UVMS (12-DOF) ---
uvms_joints_total = vehicle_joints + arm_joints
uvms_links_total  = vehicle_links + arm_links
bot_UVMS = RobotMathEngine6DOF(uvms_joints_total, uvms_links_total)
bot_UVMS.step_1_kinematics()
bot_UVMS.step_2_jacobian_M_G()
bot_UVMS.step_3_coriolis_combined()
bot_UVMS.step_4_export_cse()


1. Cinemática (Matrizes de Rotação e CMs)...
2. Dinâmica M e G (Método Jacobianos Rápido)...
   Compactando M com CSE para reuso nas derivadas...
3. Dinâmica Coriolis Total (Sem separação)...
   Calculando vetor de 12 linhas...
   Reaproveitando CSE de M nas derivadas...
   -> Linha 1 ok
   -> Linha 2 ok
   -> Linha 3 ok
   -> Linha 4 ok
   -> Linha 5 ok
   -> Linha 6 ok
   -> Linha 7 ok
   -> Linha 8 ok
   -> Linha 9 ok
   -> Linha 10 ok
   -> Linha 11 ok
   -> Linha 12 ok

 OTIMIZANDO E EXPORTANDO (CSE) 
   Compactando equações (Isso reduz o tamanho do código)...
% --- Variáveis Auxiliares ---
x0 = m10 + m11 + m12 + m3 + m4 + m5 + m6 + m7 + m8 + m9;
x1 = m2 + x0;
x2 = sin(q4);
x3 = cos(q4);
x4 = -cx4.*x2 - cy4.*x3;
x5 = cos(q5);
x6 = cx5.*x5;
x7 = sin(q5);
x8 = cz5.*x7;
x9 = -cy5.*x3 - x2.*x6 - x2.*x8;
x10 = m5.*x9;
x11 = cx6.*x5;
x12 = sin(q6);
x13 = cos(q6);
x14 = x13.*x2;
x15 = -x12.*x3 + x14.*x7;
x16 = -x15;
x17 = x13.*x3;
x18 = x12.*x2;
x19 = x17 + x18.*x7;
x20 = -x19;
x21 = cy6